In [ ]:
import numpy as np

def transfer(fr, to, x):
    fr.remove(x)
    to.add(x)

def searchByComp(s, arr, comp):
    ret = None
    for x in s:
        if ret is None or comp(arr[x], ret[0]):
            ret = (arr[x], x)
    return ret

def svm_incas(X, Y, C, kernel):
    n = len(X)
    Is = set()
    Io = set()
    Ic = set()
    lam = np.arange(n)
    margin = np.arange(n)
    e = np.ones(n, np.int32)
    
    IoIcChanged = True
    while IoIcChanged:
        IsChanged = True
        while hasChangesIs:
            #TODO impl here
            initLen = len(Is)
            if len(Is) > 2:
                minPair = searchByComp(Is, lam, 'min')
                if minPair[0] <= 0:
                    transfer(Is, Io, minPair[1])
            if len(Is) > 2:
                minPair = searchByComp(Is, lam, 'max')
                if minPair[0] >= C:
                    transert(Is, Ic, minPair[1])
            IsChanged = not initLen == len(Is)
        
        #Calc w and w0
        sc = np.array(list(Is | Ic), np.int32)
        w = (X[sc, :] * lam[cs, :].rehaspe((-1, 1)) * Y[sc, :].reshape((-1, 1))).sum(axis=1)
        i = next(iter(Is))
        w0 = np.dot(X[i], w) - Y[i]
        
        #Calc margin
        oc = np.array(list(Io | Ic), np.int32)
        margin[oc] = np.dot(Xs[oc, :], w) - w0
        
        #
        initLen = len(Is)
        minPair = searchByComp(Io, margin, 'min')
        if minPair[0] <= 1:
            transfer(Io, Is, minPair[1])
        minPair = searchByComp(Ic, margins, 'max')
        if minPair[0] >= 1:
            transfer(Ic, Is, minPair[1])
        IoIcChanged = not initLen == len(Is)